In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Basic Operation


### Install adk

In [1]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 940.5 kB/s eta 0:00:00


In [2]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [15]:
# @title Authentication to access to GCP
import sys
from IPython.display import display, Markdown

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

### Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.






In [4]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION


## Agent on adk

In [21]:
#@title Build a Agent with adk.
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types

PROMPT = """

You are a helpful assistant. Answer user questions using Google Search when needed.
Wirte a simple and clear answer to the user question.

"""

# https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
generate_content_config= types.GenerateContentConfig(
  temperature=0.2,
  max_output_tokens=8192,
)

# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.agents.LlmAgent
agent = LlmAgent(
      name="Search_assistant",
      model=MODEL_NAME,
      instruction=PROMPT,
      generate_content_config = generate_content_config,
      description="An assistant that can search the web.",
      tools=[google_search] # Use Google search tool
)


## Query to Agent

In [6]:
from google.adk.sessions import InMemorySessionService, Session

APP_NAME = "Search_Assistant"
USER_ID = "shins"

# https://google.github.io/adk-docs/sessions/
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME,
                                         user_id=USER_ID,
                                         state={"initial_key": "initial_value"})


In [22]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

# query = "What is the situation of the world economy?"
query = "트럼프 2기 행정부 이후에 세계 경제와 한국경제에 미치는 영향을 간단하게 설명해주세요.?"

# Runner : An in-memory Runner for testing and development.
# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner
runner = Runner(agent=agent,
                app_name=session.app_name,
                session_service=session_service)

# https://googleapis.github.io/python-genai/genai.html#genai.types.Content
content = types.Content(role='user', parts=[types.Part(text=query)])

# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner.run
events = runner.run(user_id=session.user_id,
                    session_id=session.id,
                    new_message=content)

for event in events:

  print(event)

  if event.is_final_response():
      final_response = event.content.parts[0].text
      display(Markdown(final_response))

      grounding_metadata = event.grounding_metadata
      print("\n\ngrounding_metadata: \n\n", grounding_metadata)

  else:
      print("No results\n")


content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="트럼프 2기 행정부가 출범할 경우 세계 경제와 한국 경제에 미치는 영향은 다음과 같이 예상될 수 있습니다.\n\n**세계 경제에 미치는 영향:**\n\n*   **보호무역주의 강화:** 트럼프 2기 행정부의 핵심 정책 중 하나는 강력한 미국 우선주의와 전면적인 보호무역주의입니다 [11]. 모든 수입품에 대한 보편 관세 부과 가능성이 있으며, 특히 중국에 대한 고율 관세 부과가 예상됩니다 [11, 15]. 이는 글로벌 무역 전쟁을 유발하고 글로벌 공급망에 불확실성을 높여 세계 경제 전반에 충격을 줄 수 있습니다 [5, 12].\n*   **경제 성장 둔화 가능성:** 미국의 강력한 관세 정책은 세계 경제 둔화의 흐름을 예상하게 합니다 [10]. 무역 갈등이 심화될 경우 세계 경제 성장률이 낮아질 수 있다는 분석도 있습니다 [10].\n*   **인플레이션 상승 압력:** 관세 부과로 인한 비용 증가는 기업들의 소비자 가격 인상으로 이어져 인플레이션을 다시 끌어올릴 수 있다는 우려가 있습니다 [5].\n*   **달러화 강세:** 트럼프 2기 정책에 대한 기대감으로 미국 주가, 국채 금리와 함께 달러화 가치가 상승하는 '트럼프 트레이드'가 나타날 수 있습니다 [5, 12]. 이는 다른 국가들의 금리 상승 부담과 달러화 부채 상환 부담 증가로 이어질 수 있습니다 [12].\n*   **주요국 정책 변화에 대한 대응 필요성:** 급변하는 주요국의 정책 변화에 대한 대응 전략 마련이 세계 경제 회복을 위한 중요한 과제로 언급됩니다 [9].\n\n**한국 경제에 미치는 영향:**\n\n*   **수출 감소 가능성:** 해외 

트럼프 2기 행정부가 출범할 경우 세계 경제와 한국 경제에 미치는 영향은 다음과 같이 예상될 수 있습니다.

**세계 경제에 미치는 영향:**

*   **보호무역주의 강화:** 트럼프 2기 행정부의 핵심 정책 중 하나는 강력한 미국 우선주의와 전면적인 보호무역주의입니다 [11]. 모든 수입품에 대한 보편 관세 부과 가능성이 있으며, 특히 중국에 대한 고율 관세 부과가 예상됩니다 [11, 15]. 이는 글로벌 무역 전쟁을 유발하고 글로벌 공급망에 불확실성을 높여 세계 경제 전반에 충격을 줄 수 있습니다 [5, 12].
*   **경제 성장 둔화 가능성:** 미국의 강력한 관세 정책은 세계 경제 둔화의 흐름을 예상하게 합니다 [10]. 무역 갈등이 심화될 경우 세계 경제 성장률이 낮아질 수 있다는 분석도 있습니다 [10].
*   **인플레이션 상승 압력:** 관세 부과로 인한 비용 증가는 기업들의 소비자 가격 인상으로 이어져 인플레이션을 다시 끌어올릴 수 있다는 우려가 있습니다 [5].
*   **달러화 강세:** 트럼프 2기 정책에 대한 기대감으로 미국 주가, 국채 금리와 함께 달러화 가치가 상승하는 '트럼프 트레이드'가 나타날 수 있습니다 [5, 12]. 이는 다른 국가들의 금리 상승 부담과 달러화 부채 상환 부담 증가로 이어질 수 있습니다 [12].
*   **주요국 정책 변화에 대한 대응 필요성:** 급변하는 주요국의 정책 변화에 대한 대응 전략 마련이 세계 경제 회복을 위한 중요한 과제로 언급됩니다 [9].

**한국 경제에 미치는 영향:**

*   **수출 감소 가능성:** 해외 의존도가 높은 한국 경제는 트럼프 2기 행정부의 통상 정책 변화에 큰 영향을 받을 것으로 예상됩니다 [6]. 미국의 관세 부과로 인해 한국의 대중국 수출 연계 생산이 감소하고 전체 수출이 줄어들 수 있다는 분석이 있습니다 [5]. 특정국 타겟 관세보다는 보편 관세 도입 여부가 한국 수출 감소의 변곡점이 될 수 있습니다 [15].
*   **산업별 영향:**
    *   **반도체:** 미국의 반도체 산업 지원 확대와 중국에 대한 기술 수출 제한은 한국 반도체 기업에 이중적인 압박으로 작용할 수 있습니다 [2].
    *   **자동차:** 인플레이션 감축법(IRA) 폐기 가능성이 시사되면서 한국 완성차 업체들이 위기감을 느낄 수 있으나, IRA가 폐기되기보다 미국 기업에 우선 혜택이 주어지고 외국 기업에 대한 혜택이 줄어들 가능성이 높다는 의견도 있습니다 [11].
*   **환율 변동성 증가:** 트럼프 2기 출범 이후 환율 변동성이 증가할 가능성이 있습니다 [2].
*   **대중국 관계 변화:** 미국과 중국의 경제 관계 축소 압박은 한국 경제 운영 방식에 직접적인 영향을 미칠 수 있습니다 [1].
*   **안보 리스크:** 경제적 영향 외에 안보 측면에서도 한국에 대한 역할 확대 요구 등 '안보 쓰나미'가 올 수 있다는 전망도 있습니다 [1].
*   **기회 요인:** 미국의 강한 성장세는 한국 경제 성장에도 긍정적인 영향을 줄 수 있으며, 환경/에너지 분야 규제 완화로 단기적인 수혜를 볼 가능성도 있습니다 [13]. 또한, 한미 협상의 여지와 기회 요인도 존재할 수 있습니다 [8].

종합적으로 트럼프 2기 행정부 출범은 보호무역주의 강화와 예측 불가능한 정책 변화로 인해 세계 경제와 한국 경제에 상당한 불확실성과 하방 위험을 높일 것으로 예상됩니다 [3, 8, 11]. 한국 경제는 수출 부진 심화, 특정 산업의 어려움 가중 등 부정적인 영향을 받을 가능성이 크지만, 일부 긍정적인 영향이나 기회 요인도 있을 수 있습니다 [8, 13]. 이에 대한 선제적인 대비와 유연한 대응 전략 마련이 중요합니다 [1, 8].



grounding_metadata: 

 grounding_chunks=[GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain='fki.or.kr', title='fki.or.kr', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAIOZuF3rvmvmCckwKGjhxHWorVRXDdBydasvaG2emw3PqUEy6OAQTKrwmNvnRdA-gOXipO4XOPe6zQRdtS6rPRR4PafFuq12oI03UJq0HkBsP538FXCgGH0Q1UZCx4JoKVvc0T9BU0I9LQ=')), GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain='kdi.re.kr', title='kdi.re.kr', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJOtQHhYuUQVj8o5rv6VvucLS7O0-2BmfbcAbl1DyScEnszMwQX5Zpdne5KScJSTXxpWDoyVf6dxJ9FQGKskZtrXWdBg7LCFN0XeNXBG6QqURqLNBwCdNtaniVgV9nL6c4M')), GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain='bok.or.kr', title='bok.or.kr', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJr05zBTENrQIkfxt4PJke7rvEJzX6eXEPUSaxhw70wEWmxbw6405G8TQkZlV1FF7GtIx2Zu39Lrm3wFzPALkUHxqxj3Sx84pBikd745p2O5C6cZgaBIhIs_9gBxOJHVog7YABfmmWg4icIpPi_OtEIoB-

## End of notebook